In [1]:
from langame import LangameClient
import json
import openai
import datetime
c = LangameClient("../config.yaml")

# topic classification

In [ ]:
file_name = "classification-topic-endpoint-0.0.1.jsonl"
with open(file_name, 'w') as outfile:
    for e in c._firestore_client.collection("memes").stream():
        json.dump({
            "text": e.get("content"),
            "label": e.get("topics")[0]
        }, outfile)
        outfile.write('\n')

In [ ]:
c_f = openai.File.create(
    file=open(file_name),
    purpose="classifications"
)

In [ ]:
openai.Classification.create(
    file="file-rV9rZbiiziXLfjqc4MKLLChy",
    query="123",
    search_model="ada", 
    model="curie", 
    max_examples=3
)

# quality classification

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("classification")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage


Using 16 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 02/08/2022, 20:17:52
There are 635 embeddings of dim 768
	Compute estimated construction time of the index 02/08/2022, 20:17:52
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0000 secs
	Checking that your have enough memory available to create the index 02/08/2022, 20:17:52
2.2MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0004 secs
	Selecting most promising index types given data characteristics 02/08/2022, 20:17:52
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 02/08/2022, 20:17:52
		-> Instanciate the index Flat 02/08/2022, 20:17:52
		>>> Finished "-> Instanciate the index Flat" in 0.0008 sec

100%|██████████| 1/1 [00:00<00:00, 947.44it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0028 secs
	>>> Finished "Creating the index" in 0.0043 secs
	Computing best hyperparameters 02/08/2022, 20:17:52
	>>> Finished "Computing best hyperparameters" in 0.0000 secs
The best hyperparameters are: 
	Compute fast metrics 02/08/2022, 20:17:52


2540
	>>> Finished "Compute fast metrics" in 1.0483 secs
	Saving the index on local disk 02/08/2022, 20:17:53
	>>> Finished "Saving the index on local disk" in 0.0014 secs
Recap:
{'99p_search_speed_ms': 0.4987652366980912,
 'avg_search_speed_ms': 0.4106569778555491,
 'compression ratio': 0.9999769321266272,
 'index_key': 'Flat',
 'index_param': '',
 'nb vectors': 635,
 'reconstruction error %': 0.0,
 'size in bytes': 1950765,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 1.0619 secs


In [9]:
out_file_name = f"../data/fine_tune_fitness_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ### {e['content']} ~~~",
            "completion": f" 1\n",
        }, outfile)
        outfile.write('\n')
for e in c._firestore_client.collection("deleted_memes").stream():
    # check has "topics" and "content"
    if "topics" not in e.to_dict() or "content" not in e.to_dict():
        continue
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e.get('topics'))} ### {e.get('content')} ~~~",
            "completion": f" 0\n",
        }, outfile)
        outfile.write('\n')

In [13]:
!head -n3 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "ice breaker ### When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "ecology ### Have natural disasters gotten worse with the increase in human existence? If so, why? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "ice breaker ### What would prompt you to make a dramatic change in your life. Is it worth it? ~~~",
  "completion": " 1\n"
}


In [8]:
!openai tools fine_tunes.prepare_data -f $out_file_name

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Analyzing...

- Your file contains 1641 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 2 duplicated prompt-completion sets. These are rows: [402, 1617]
- All prompts end with suffix ` ###`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 2 duplicate rows [Y/n]: ^C



In [15]:
openai_file = openai.File.create(
  file=open(out_file_name),
  purpose="fine-tune"
)
openai.FineTune.create(
    training_file=openai_file["id"],
    model="ada",
    # "On classification tasks, we recommend setting this to false"
    use_packing=False,
    # https://beta.openai.com/docs/api-reference/fine-tunes/create#fine-tunes/create-prompt_loss_weight
    # classification recommended 0.1 (it's the default value anyway)
    prompt_loss_weight=0.1
)

<FineTune fine-tune id=ft-AibnxdtbRb0PCJzL17IggXTW at 0x7fca3c58bf90> JSON: {
  "created_at": 1644348707,
  "events": [
    {
      "created_at": 1644348707,
      "level": "info",
      "message": "Created fine-tune: ft-AibnxdtbRb0PCJzL17IggXTW",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-AibnxdtbRb0PCJzL17IggXTW",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 250247,
      "created_at": 1644348706,
      "filename": "fine_tune_fitness_08_02_2022.jsonl",
      "id": "file-U8kr1va0RfK0yVjAA4dR97wo",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1644348707,
  "validation_files": []
}